# Blackjack
### Rules:
- Maximum number of players is 6. If the given number exceeds 6 players, the number of players is set to 6.
- By default, each player is given 100 dollars.
- Minimum bet is 20 dollars.
- Jack, Queen, and King are treated as equivalents of 10 when splitting.
- A player can split up to three times in a round, resulting in four hands maximum. 
- A player cannot split aces.
- A 21 after split is not considered Blackjack.
- You earn 1.5 times your bet for Blackjack.
- No surrender.
- No insurance.
- If a player chooses not to play for the given round, the result will be shown 'lose' but the total amount of money will remain unchanged.
- Game ends when there are no players left.
- It is assumed that the number inputs are given in non-negative integers.

Refer to https://en.wikipedia.org/wiki/Blackjack for other basic Blackjack rules.

In [8]:
# import libraries
import numpy as np
import pandas as pd
import random
import math

In [13]:
class Player:
    def __init__(self):
        self.hand = [[]] # hand
        self.score = [0] # score for each hand
        self.bet = [] # money bet
        self.deal = [True] # True until stand or bust
        self.money = 100 # total money left
        self.sh = 'None' # soft/hard hand
        self.blackjack = False # True if blackjack
        self.bonus=0 # bonus for blackjack
        self.play = True # whether or not to play this round
    
    # player bets money
    def betmoney(self, bet_amount):
        left = self.money - bet_amount
        if left >= 0:
            self.money = left
            self.bet.append(bet_amount)
        return left
    
    # add score for the given card
    def addscore(self, i, card):
        if card in ['J', 'Q', 'K']: card = 10 # treat J, Q, K as 10
        if card == 'A': # soft hand
            card = 11
            self.sh = 'soft'
        score = self.score[i] + card
        if (score > 21) & (self.sh == 'soft'): # hard hand
            score -= 10
            self.sh = 'hard'
        if score==21: self.deal[i]=False # stop dealing when score==21
        if score > 21: # stop dealing when score>21
            self.score[i] = -1 # set score=-1 when busted
            self.deal[i] = False
        else:
            self.score[i] = score
    
    # deal a card for the player
    def hit(self, i, deck):
        card = deck.pop()
        self.hand[i].append(card)
        self.addscore(i, card)
    
    # stop dealing more cards for the player
    def stand(self, i):
        self.deal[i] = False
    
    # split two same cards into two hands
    def split(self, i):
        hand = [10 if (x in ['J', 'Q', 'K']) else x for x in self.hand[i]] # treat J, Q, K as 10
        if (len(hand)==2): # only split when two cards are given
            if hand[0]==hand[1]!='A': # only two identical cards excluding aces can be splitted
                if(len(self.hand)>=4): # split up to 3 times
                    print('You can split only up to 3 times.')
                    return
                if self.betmoney(self.bet[0]) >= 0: # check if the player has enough money
                    self.hand.append([self.hand[i].pop()])
                    self.deal.append(True)
                    self.score[i] = int(self.score[i]/2)
                    self.score.append(self.score[i])
                    return
                print('You don\'t have enough money.')
                return
        print('You cannot split.')
    
    # double the bet and deal only one more card
    def doubledown(self, i, deck):
        if self.bet[i]>self.money: # check if the player has enough money
            print('You don\'t have enough money.')
            return
        self.money -= self.bet[i]
        self.bet[i] *= 2
        self.hit(i, deck)
        self.deal[i] = False
        
    # player gets blackjack
    def bj(self):
        self.blackjack=True
        self.bonus = math.floor(self.bet[0]*0.5)
    
    # print player's hand
    def printhand(self, i):
        print('Your hand: {}'.format(self.hand[i]))
        score = self.score[i]
        if score == -1:
            print('Busted!')
        else:
            print('Your current score for this hand is {}'.format(score))
    
    # add money earned
    def addmoney(self, amount):
        self.money+=amount
    
    # reset hands and scores
    def reset(self):
        self.hand = [[]]
        self.score = [0]
        self.bet = []
        self.deal = [True]
        self.sh = 'None'
        self.blackjack = False
        self.bonus = 0
        self.play = True
   

        
class Dealer:
    def __init__(self):
        self.hand = [] # dealer hand
        self.score = 0 # dealer score
        self.sh = 'None' # soft/hard hand
        self.blackjack = False # True if blackjack
    
    # add score from the given card
    def addscore(self, card):
        if card in ['J', 'Q', 'K']: card = 10 # treat J, Q, K as 10
        if card == 'A': # soft hand
            card = 11
            self.sh = 'soft'
        score = self.score + card
        if (score > 21) & (self.sh == 'soft'): # hard hand
            score -= 10
            self.sh = 'hard'
        self.score = score    
        
    # deal a new card for the dealer
    def deal(self, deck):
        card = deck.pop()
        self.hand.append(card)
        self.addscore(card) 
        
    # reset the dealer score and hand
    def reset(self):
        self.hand=[]
        self.score=0
        self.sh = 'None'
        self.blackjack = False

In [14]:
# player decides whether or not to quit the game
def quitornot(name):
    while(True): # repeat until the user gives a valid input
        quit = input(name + ', would you like to quit? (y/n)\n').lower()
        if quit=='y':
            return True
        elif quit=='n':
            return False
        print('Invalid input.')
        
        
        
# player decides whether or not to play this round
def playornot(name, player):
    while(True): # repeat until the user gives a valid input
        play = input(name + ', would you like to play this round? (y/n)\n').lower()
        if play=='y': 
            return True
        elif play=='n':
            player.play = False
            return False
        print('Invalid input')

In [19]:
def game():
    print("Welcome to Blackjack!")
    
    # initialize players dictionary and dealer
    n_players = 0
    d = {}
    dealer = Dealer()
    
    # create six decks of cards and shuffle
    deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'] * 24
    print('Shuffling...')
    random.shuffle(deck)
    
    # play the game
    while(True):
        # shuffle when the number of cards left undealt is less than 75
        if (len(deck)<75):
            deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'] * 24
            print('Shuffling...')
            random.shuffle(deck)
        
        # initialize new players (maximum of 6 total number of players)
        if len(d)<6:
            new_p = int(input('How many new players want to join this round? (e.g. 3) \n'))
            if new_p > 6-len(d): new_p = 6-len(d)
            for n in range(n_players, n_players+new_p):
                d['Player {}'.format(n)] = Player()
            n_players += new_p
        
        # record if each player is playing/quitting
        quitting = []
        playing = []
        for name, player in d.items():
            play = playornot(name, player)
            if play: playing.append(name)
            else: # skip the betting part that follows if the player chooses not to play this round
                quit = quitornot(name)
                if quit: quitting.append(name)
                continue
            
            # Let players who are playing this round bet.
            bet = True
            while(bet):
                bet_amount = int(input('How much would you like to bet? \n'))
                if bet_amount < 20:
                    print('Minimum bet is 20 dollars.')
                    continue
                left = player.betmoney(bet_amount)
                if left < 0:
                    print('You don\'t have enough money. Try smaller amount.')
                    continue
                bet = False
        
        # if none of the players are playing, go to the next round or end the game if no players are left
        if len(playing) == 0: 
            for name in quitting: d.pop(name)
            if len(d)==0: break
            for name, player in d.items():
                player.reset()
            continue
        
        # deal two cards for the dealer
        dealer.deal(deck)
        dealer.deal(deck)    
        print('The dealer is dealt two cards. The first card is {}.'.format(dealer.hand[0]))
        
        # each player plays the game by choosing one of hit, stand, split, and double down
        for name, player in d.items():
            if not player.play: continue # skip if the player is not playing this round
            i=0
            for hand in player.hand:
                print('{}, your hand is {}'.format(name, hand))
                while player.deal[i]:
                    action = input('Enter one of the following--hit, stand, split, double down: \n').lower()
                    if action=='hit':
                        player.hit(i, deck)
                        player.printhand(i)
                        continue
                    if action=='stand':
                        player.stand(i)
                        player.printhand(i)
                        continue
                    if action=='split':
                        player.split(i)
                        player.printhand(i)
                        continue
                    if action=='double down':
                        player.doubledown(i, deck)
                        player.printhand(i)
                        continue
                i+=1
            
            # check if the user got Blackjack
            if (len(player.hand)==1) & (len(player.hand[0])==2) & (player.score[0]==21):
                print('BLACKJACK!')
                player.bj()
        
        # remove users who chose to quit
        for name in quitting: d.pop(name)
        
        # deal more cards for the dealer until the score reaches 17 or more and print the result
        while dealer.score<17:
            dealer.deal(deck)
        print('Dealer hand: {}'.format(dealer.hand))
        if dealer.score<=21:
            print('Dealer score: {}'.format(dealer.score))
        else:
            print('Dealer busted!')
            dealer.score=0
        if (len(dealer.hand)==2) & (dealer.score==21):
            print('BLACKJACK!')
            dealer.blackjack = True
        
        # get results for each user
        for name, player in d.items():
            if dealer.blackjack:
                result = ['push' if player.blackjack else 'lose' for x in player.score]
                reward = [b if r=='push' else 0 for r, b in zip(result, player.bet)]
            else:
                result = ['win' if x>dealer.score or player.blackjack else 'push' if x==dealer.score else 'lose' for x in player.score]
                reward = [b*2 + player.bonus if r=='win' else b if r=='push' else 0 for r, b in zip(result, player.bet)]
            player.addmoney(sum(reward))
            print(name+', your result is {}'.format(result)+' and your total earn is {}.'.format(sum(reward)-sum(player.bet))
                 + ' You have {} left.'.format(player.money))
        
        # remove users who lost all the money
        lost = []
        for name, player in d.items():
            if player.money <= 0:
                print('{} lost all the money'.format(name))
                lost.append(name)
        for name in lost: d.pop(name)
        
        # stop the game if there are no players left
        if len(d)==0: break
        
        # reset hands and scores before starting the next round
        for name, player in d.items():
            player.reset()
        dealer.reset()
    
    print('No players left. Game ended.')

In [20]:
game()

Welcome to Blackjack!
Shuffling...
How many new players want to join this round? (e.g. 3) 
3
Player 0, would you like to play this round? (y/n)
y
How much would you like to bet? 
20
Player 1, would you like to play this round? (y/n)
y
How much would you like to bet? 
20
Player 2, would you like to play this round? (y/n)
y
How much would you like to bet? 
20
The dealer is dealt two cards. The first card is 6.
Player 0, your hand is []
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: ['J']
Your current score for this hand is 10
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: ['J', 'J']
Your current score for this hand is 20
Enter one of the following--hit, stand, split, double down: 
split
Your hand: ['J']
Your current score for this hand is 10
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: ['J', 4]
Your current score for this hand is 14
Enter one of the following--hit, stand, split, double down: 
hit
You

Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [5, 10, 'K']
Busted!
Player 3, your hand is []
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [10]
Your current score for this hand is 10
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [10, 3]
Your current score for this hand is 13
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [10, 3, 9]
Busted!
Player 4, your hand is []
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [5]
Your current score for this hand is 5
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [5, 3]
Your current score for this hand is 8
Enter one of the following--hit, stand, split, double down: 
hit
Your hand: [5, 3, 'Q']
Your current score for this hand is 18
Enter one of the following--hit, stand, split, double down: 
stand
Your hand: [5, 3, 'Q']
Your current score for this hand is 18
Player 5, your han